## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-12-21-50-30 +0000,nyt,New Epstein Emails Alleged Trump Knew of His C...,https://www.nytimes.com/2025/11/12/us/politics...
1,2025-11-12-21-48-09 +0000,nypost,Costco recalls nearly 1M bottles of Kirkland p...,https://nypost.com/2025/11/12/business/costco-...
2,2025-11-12-21-46-00 +0000,wsj,Opinion | Is ‘Trump Derangement Syndrome’ Real?,https://www.wsj.com/opinion/is-trump-derangeme...
3,2025-11-12-21-45-20 +0000,nyt,Epstein Bantered Regularly With Larry Summers,https://www.nytimes.com/2025/11/12/us/politics...
4,2025-11-12-21-44-46 +0000,nyt,"As Consumer Bureau’s Cash Dwindles, Trump Admi...",https://www.nytimes.com/2025/11/12/business/co...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2349/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
4,trump,39
70,shutdown,18
0,new,17
67,government,15
1,epstein,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
191,2025-11-12-13-56-28 +0000,nypost,Furious White House reveals Jeffrey Epstein vi...,https://nypost.com/2025/11/12/us-news/jeffrey-...,116
355,2025-11-11-22-13-00 +0000,wsj,Senate Minority Leader Chuck Schumer wasn’t on...,https://www.wsj.com/politics/policy/progressiv...,97
79,2025-11-12-19-28-42 +0000,nypost,Michael Wolff told Epstein he should try to en...,https://nypost.com/2025/11/12/us-news/michael-...,96
213,2025-11-12-11-42-45 +0000,nypost,Zohran Mamdani says he’ll reach out to Trump b...,https://nypost.com/2025/11/12/us-news/zohran-m...,88
61,2025-11-12-20-02-00 +0000,wsj,Democrats Release New Epstein Emails Referring...,https://www.wsj.com/politics/policy/democrats-...,87


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
191,116,2025-11-12-13-56-28 +0000,nypost,Furious White House reveals Jeffrey Epstein vi...,https://nypost.com/2025/11/12/us-news/jeffrey-...
311,85,2025-11-12-02-00-00 +0000,wsj,As data went dark from the government shutdown...,https://www.wsj.com/economy/central-banking/th...
29,62,2025-11-12-21-02-48 +0000,nypost,Ex-top NY aide accused of being secret agent f...,https://nypost.com/2025/11/12/us-news/ex-top-n...
194,54,2025-11-12-13-51-00 +0000,nyt,India’s Prime Minister Vows Justice After New ...,https://www.nytimes.com/2025/11/11/world/asia/...
57,48,2025-11-12-20-09-00 +0000,wsj,Atlanta Fed President Raphael Bostic will reti...,https://www.wsj.com/economy/central-banking/at...
279,44,2025-11-12-07-18-55 +0000,nypost,Marriott guests left ‘homeless’ as rental comp...,https://nypost.com/2025/11/12/business/marriot...
295,41,2025-11-12-03-32-40 +0000,nypost,Bank of America CEO Brian Moynihan seeks ‘impo...,https://nypost.com/2025/11/11/business/bank-of...
355,40,2025-11-11-22-13-00 +0000,wsj,Senate Minority Leader Chuck Schumer wasn’t on...,https://www.wsj.com/politics/policy/progressiv...
94,37,2025-11-12-18-48-19 +0000,wapo,Turkish military plane falls from sky over Geo...,https://www.washingtonpost.com/world/2025/11/1...
276,37,2025-11-12-07-26-25 +0000,nypost,Chess federation files complaint against ex-wo...,https://nypost.com/2025/11/12/world-news/chess...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
